In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_csv("drive/MyDrive/statlearning/merged_data.csv", encoding='utf-8')
df.head()

,url,publish_date,author,content
0,https://vietstock.vn/2025/05/con-gai-bau-duc-g...,2025-05-17 12:58:00,Thế Mạnh,Con gái bầu Đức gom bất thành 4 triệu cp HAG v...
1,https://vietstock.vn/2025/05/hose-thang-4-hau-...,2025-05-12 15:39:00,Huy Khải,"HOSE tháng 4: Hầu hết chỉ số ngành giảm điểm, ..."
2,https://vietstock.vn/2025/05/dau-tu-phat-trien...,2025-05-02 07:22:00,Thu Minh,Đầu tư Phát triển Mỹ Khánh tất toán trước hạn ...
3,https://vietstock.vn/2025/05/thanh-khoan-upcom...,2025-05-09 15:32:00,Huy Khải,Thanh khoản UPC oM tăng 13% trong tháng 4.Thị ...
4,https://vietstock.vn/2025/05/thi-truong-chung-...,2025-05-18 18:58:00,Bộ phận Phân tích Kinh tế & Chiến lược Thị trư...,Thị trường chứng quyền tuần 19-23/05/2025: Sắc...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6452 entries, 0 to 6451
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           6452 non-null   object
 1   publish_date  6452 non-null   object
 2   author        6452 non-null   object
 3   content       6452 non-null   object
dtypes: object(4)
memory usage: 201.8+ KB


In [4]:
df['title'] = (
    df['content']
      .str.split(r'[\.…]', n=1, expand=True)   # split theo . hoặc …, chỉ việc 1 lần
      .iloc[:, 0]                              # lấy phần trước dấu
      .str.strip()                             # xóa khoảng trắng thừa
)

In [5]:
df['publish_date'] = (
    pd.to_datetime(df['publish_date'])
      .dt.date
      .astype(str)   # để lưu thành chuỗi "2025-05-17"
)

In [6]:
df.head()

,url,publish_date,author,content,title
0,https://vietstock.vn/2025/05/con-gai-bau-duc-g...,2025-05-17,Thế Mạnh,Con gái bầu Đức gom bất thành 4 triệu cp HAG v...,Con gái bầu Đức gom bất thành 4 triệu cp HAG v...
1,https://vietstock.vn/2025/05/hose-thang-4-hau-...,2025-05-12,Huy Khải,"HOSE tháng 4: Hầu hết chỉ số ngành giảm điểm, ...","HOSE tháng 4: Hầu hết chỉ số ngành giảm điểm, ..."
2,https://vietstock.vn/2025/05/dau-tu-phat-trien...,2025-05-02,Thu Minh,Đầu tư Phát triển Mỹ Khánh tất toán trước hạn ...,Đầu tư Phát triển Mỹ Khánh tất toán trước hạn ...
3,https://vietstock.vn/2025/05/thanh-khoan-upcom...,2025-05-09,Huy Khải,Thanh khoản UPC oM tăng 13% trong tháng 4.Thị ...,Thanh khoản UPC oM tăng 13% trong tháng 4
4,https://vietstock.vn/2025/05/thi-truong-chung-...,2025-05-18,Bộ phận Phân tích Kinh tế & Chiến lược Thị trư...,Thị trường chứng quyền tuần 19-23/05/2025: Sắc...,Thị trường chứng quyền tuần 19-23/05/2025: Sắc...


In [7]:
import sqlite3

In [8]:
def fill_metadata_from_df(c, df):
    """
    Với mỗi dòng trong df, cập nhật title và publish_date
    cho record tương ứng trong bảng Articles dựa trên source_url.
    """
    for _, row in df.iterrows():
        url   = row['url']
        title = row['title']
        date  = row['publish_date']  # định dạng "YYYY-MM-DD"

        # Chỉ UPDATE chứ không chèn mới
        c.execute("""
            UPDATE Articles
            SET title = ?, publish_date = ?
            WHERE source_url = ?
        """, (title, date, url))

In [9]:
conn = sqlite3.connect('stock_insights.db')
c = conn.cursor()

fill_metadata_from_df(c, df)
print("Done ...")

# 4. Lưu và đóng
conn.commit()
conn.close()

Done ...


In [10]:


conn = sqlite3.connect('stock_insights.db')
c = conn.cursor()

# Tổng số bài
c.execute("SELECT COUNT(*) FROM Articles")
total = c.fetchone()[0]

# Số bài chưa cập nhật title
c.execute("SELECT COUNT(*) FROM Articles WHERE title='' OR title IS NULL")
no_title = c.fetchone()[0]

# Số bài chưa cập nhật publish_date
c.execute("SELECT COUNT(*) FROM Articles WHERE publish_date='' OR publish_date IS NULL")
no_date = c.fetchone()[0]

print(f"Tổng bài: {total}")
print(f"Bài chưa có title: {no_title}")
print(f"Bài chưa có publish_date: {no_date}")
print("\n--- 5 bản ghi đầu (URL, title, publish_date) ---")
for row in c.execute("""
    SELECT source_url, title, publish_date
    FROM Articles
    ORDER BY article_id
    LIMIT 5
"""):
    print(row)

conn.close()


Tổng bài: 6450
Bài chưa có title: 0
Bài chưa có publish_date: 0

--- 5 bản ghi đầu (URL, title, publish_date) ---
('https://vietstock.vn/2025/03/nhom-co-lien-quan-den-beston-tang-so-huu-tai-vib-len-hon-8-739-1284968.htm', 'Nhóm có liên quan đến Beston tăng sở hữu tại VIB lên hơn 8%', '2025-03-24')
('https://vietstock.vn/2025/03/lanh-dao-may-song-hong-muon-rut-sach-von-sau-khi-co-phieu-lap-dinh-739-1283999.htm', 'Lãnh đạo May Sông Hồng muốn rút sạch vốn sau khi cổ phiếu lập đỉnh', '2025-03-21')
('https://vietstock.vn/2025/04/co-phieu-nao-keo-vn-index-bat-lai-sau-cu-roi-hon-200-diem-830-1294791.htm', 'Cổ phiếu nào kéo VN-Index bật lại sau cú rơi hơn 200 điểm?', '2025-04-12')
('https://vietstock.vn/2024/02/co-phieu-hng-co-kha-nang-bi-huy-niem-yet-bat-buoc-do-thua-lo-lien-tiep-3-nam-741-1155036.htm', 'Cổ phiếu HNG có khả năng bị hủy niêm yết bắt buộc do thua lỗ liên tiếp 3 năm', '2025-06-10')
('https://vietstock.vn/2025/02/quy-dau-tu-manh-tay-mua-co-phieu-3358-1274221.htm', 'Quỹ đầu tư mạn